In [1]:
import pandas as pd
import numpy as np

import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

In [2]:
df = pd.read_csv('Data/cleaned_data_2.csv')

In [3]:
df

,Head,Body,Tags,First Answer,Tags Count,Text,Text_Uncleaned,Text_Cleaned,Head_Cleaned,Body_Cleaned
0,brain segmentation to 3d model,my goal is to take a dataset of brain tumor se...,computer-vision python,No Answer,2,brain segmentation to 3d model my goal is to t...,brain segmentation 3d model goal dataset brain...,brain segmentation 3d model goal dataset brain...,brain segmentation 3d model,goal dataset brain tumor segmentation use comp...
1,active learning regression with random forest,i have a dataset of about 8k points and i am t...,machine-learning regression uncertainty-quanti...,No Answer,5,active learning regression with random forest ...,active learning regression random forest datas...,active learn regression random forest dataset ...,active learning regression random forest,dataset 8k point try employ active learn rando...
2,comparing reinforcement learning models,i am currently completing my thesis on optimis...,reinforcement-learning policy-gradients monte-...,No Answer,4,comparing reinforcement learning models i am c...,comparing reinforcement learning models curren...,compare reinforcement learning model currently...,compare reinforcement learning model,currently complete thesis optimise combinatori...
3,why good model that performs great on holdout ...,i have this binary regression model that has 5...,deep-learning deep-neural-networks prediction ...,No Answer,5,why good model that performs great on holdout ...,good model performs great holdout validation d...,good model perform great holdout validation da...,good model perform great holdout validation da...,binary regression model 500 future unbalanced ...
4,what are reservoir computers used for today,reservoir computers were very popular in the e...,machine-learning recurrent-neural-networks res...,No Answer,3,what are reservoir computers used for today r...,reservoir computers today reservoir computers ...,reservoir computer today reservoir computer po...,reservoir computer today,reservoir computer popular early 2000 understa...
...,...,...,...,...,...,...,...,...,...,...
48806,where to get older digital ocr'd data sets of ...,i am trying to locate older unsummarized us ce...,data-request usa us-census,No Answer,3,where to get older digital ocr'd data sets of ...,older digital ocr'd data sets unsummarized cen...,old digital ocr'd datum set unsummarized censu...,old digital ocr'd datum set unsummarized censu...,trying locate old unsummarized census datum re...
48807,orthophoto rwanda free download,i'm looking for image datas for east africa sp...,data-request images aerial-photography africa,No Answer,4,orthophoto rwanda free download i'm looking f...,orthophoto rwanda free download looking image ...,orthophoto rwanda free download look image dat...,orthophoto rwanda free download,look image data east africa specifically rwand...
48808,finding online finance datasets,i am searching for finance datasets that has t...,data-request finance,No Answer,2,finding online finance datasets i am searching...,finding online finance datasets searching fina...,find online finance dataset search finance dat...,find online finance dataset,search finance dataset follow format year quar...
48809,healthcare finder api links broken,it seems that the healthcare finder api schema...,healthcare-finder-api,Yes this looks like a bug on the site. The API...,1,healthcare finder api links broken it seems t...,healthcare finder api links broken healthcare ...,healthcare finder api link break healthcare fi...,healthcare finder api link break,healthcare finder api schema link long work ht...


In [4]:
# Assuming your dataset has 'title', 'body', and 'tags' columns
X = df['Text_Cleaned']
y = [tags.split() for tags in df['Tags']]  # Assuming tags are space-separated strings

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and train Word2Vec model
sentences = [text.split() for text in X_train]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)
print('Word2Vec model training complete.')

Word2Vec model training complete.


In [5]:
# Vectorize the training data
def question_vectorizer(question, model):
    tokens = question.split()
    word_vectors = [model.wv[token] for token in tokens if token in model.wv]
    if not word_vectors:
        # Handle the case where no tokens are in the vocabulary
        return None
    # Calculate the average of word vectors
    avg_vector = sum(word_vectors) / len(word_vectors)
    return avg_vector

X_train_vec = [question_vectorizer(question, model) for question in X_train]
X_test_vec = [question_vectorizer(question, model) for question in X_test]

In [6]:
# Multi-label binarization for target variable
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)

d:\Github\Automatic-Question-Tagging-System\.venv\lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['a', 'adadelta', 'adversarial-example', 'aevb-algorithm', 'allennlp', 'arfima', 'b', 'boxcox-transformation', 'case-cohort', 'cifar-10', 'codon', 'collecting-data', 'conjunctive-normal-form', 'consulting', 'cosinor', 'counts', 'critical-value', 'cubic', 'customer-lifetime-value', 'death', 'decision-rule', 'diagnosis', 'document-term-matrix', 'dose-response', 'e', 'elastic-weight-consolidation', 'electric', 'energy-based-models', 'exploratory-factor-analysis', 'functional-api', 'fuzzy-classification', 'gas', 'gauss-markov-theorem', 'goodman-kruskal-gamma', 'grammatical-evolution', 'graphviz', 'i', 'image-restoration', 'infrastructure', 'instruct-gpt', 'l', 'latent-dirichlet-allocation', 'lifelines', 'lime', 'linear-dynamical-system', 'logarithmic-series-distribution', 'lorenz-curve', 'lpips', 'lunar-lander', 'market-basket-analysis', 'mc-dropout', 'met

In [7]:
# Making a list of model best suited for text and multitags classification

models = [('RandomForestClassifier', RandomForestClassifier()),
        ('SGDClassifier', SGDClassifier()),
        ('LogisticRegression', LogisticRegression()),
        ('LinearSVC', LinearSVC()),
        ('MultinomialNB', MultinomialNB()),
        ('KNeighborsClassifier', KNeighborsClassifier()),
        ('OneVsRestClassifier', OneVsRestClassifier(LinearSVC()))]

In [13]:
# Training and testing the models using cross validation

from sklearn.model_selection import cross_val_score,StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for name, model in models:
    try:
        cv_results = cross_val_score(model, np.array(X_train_vec), y_train_bin, cv=kfold, scoring='f1_micro')
        print(f'{name}: {cv_results.mean()} ({cv_results.std()})')
    except:
        print(f'{name}: Error')

RandomForestClassifier: Error
SGDClassifier: Error
LogisticRegression: Error
LinearSVC: Error
MultinomialNB: Error
KNeighborsClassifier: Error
OneVsRestClassifier: Error
